# Import the Libraries

In [ ]:
import os
import pandas as pd
import numpy as np

from bokeh.plotting import figure, show, save
from bokeh.io import output_file
from bokeh.models import ColumnDataSource, HoverTool, DateFormatter
from bokeh.models.widgets import DateRangeSlider
from bokeh.layouts import row, widgetbox

import holoviews as hv
from holoviews import opts
from holoviews.plotting.links import RangeToolLink

import panel as pn

hv.extension('bokeh')

# Import the Data and construct the DataFrame

In [ ]:
dataframe = pd.read_csv('Deposits_and_Inflation_Data.csv')

In [ ]:
country = 'Portugal'

In [ ]:
chosen_columns = []
for column in dataframe.columns:
    if dataframe.iloc[0][column] == country:
        chosen_columns.append(dataframe[column])
joint_table = pd.concat(chosen_columns, axis=1)

In [ ]:
joint_table['Code'] = dataframe['Code']

country_table = joint_table[3:]
country_table['Code'] = pd.to_datetime(country_table['Code'])
country_table.columns = joint_table.iloc[1]
country_table = country_table.rename(columns = {"Metric":"Date", "Inflation rate": "Inflation_rate", 
                                                "Interest rate":"Interest_rate", "Real Interest rate": "Real_Interest_rate"})
country_table = country_table.set_index('Date')
country_table = country_table.apply(pd.to_numeric)

# Graphs - Holoviews, based on Bokeh

## Graph 1 - Holoviews with RangeTool

In [ ]:
#Generate all curves
def getCurves(n):
    for column in country_table.columns:
        curve = hv.Curve(country_table[column], label = column)
        yield curve

x_range = (country_table['Interest_rate'].replace(0, np.nan).first_valid_index(), country_table['Interest_rate'].index[-1])
y_range = (country_table.min().min() - 0.5, country_table.max().max() + 0.5)
        
source_curves, target_curves  = [], []
for curve in getCurves(10):
    src = curve.relabel('').opts(width=800, height=100, yaxis=None, default_tools=[], xlim = x_range, ylim = y_range) 
    tgt = curve.opts(opts.Curve(tools=['hover'], width=800, xlim = x_range, ylim = y_range, ylabel = 'Percentage'))
    source_curves.append(src)
    target_curves.append(tgt)     

# link RangeTool for the first curves in the list.
RangeToolLink(source_curves[0],target_curves[0])

#Overlay the source and target curves
overlaid_plot_src = hv.Overlay(source_curves).relabel('')    
overlaid_plot_tgt = hv.Overlay(target_curves).relabel('Portugal').opts(height=400)

# layout the plot and render
layout = (overlaid_plot_tgt + overlaid_plot_src).cols(1)
layout = layout.opts(merge_tools=False,shared_axes=False)

#Save the Plot
p = pn.panel(layout)
p.save('Graph_1.html', embed = True)

## Graph 2 - Holoviews with DropDown List

### Constructing a new DataFrame

In [ ]:
variable = 'Real Interest rate'

In [ ]:
chosen_columns = []
for column in dataframe.columns:
    if dataframe.iloc[1][column] == variable:
        chosen_columns.append(dataframe[column])
joint_table = pd.concat(chosen_columns, axis=1)

In [ ]:
joint_table['Code'] = dataframe['Code']

variable_table = joint_table[3:]
variable_table['Code'] = pd.to_datetime(variable_table['Code'])
variable_table.columns = joint_table.iloc[0]
variable_table = variable_table.rename(columns = {"Country":"Date"})
variable_table = variable_table.set_index('Date')
variable_table = variable_table.apply(pd.to_numeric)

### Deploying the Graph

In [ ]:
#Include Dropdown List
def load_countries(Country): 
    curve = hv.Curve(variable_table[Country])
    curve = curve.opts(opts.Curve(tools=['hover'], width=700, ylabel = 'Real Interest rate, in %'))
    return curve
        
countries_name = variable_table.columns

dmap = hv.DynamicMap(load_countries, kdims='Country').redim.values(Country=countries_name)

dmap = dmap.opts(framewise=True)

#Save the Plot - Use Panel package
p = pn.panel(dmap)
p.save('Graph_2.html', embed = True)